#Imports and functions

In [1]:
"""
from google.colab import drive
drive.mount('/content/drive')
"""

Mounted at /content/drive


In [2]:
!pip install captum
!pip install transformers
!pip install pybedtools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 75.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 68.3 MB/s eta 0:00:00
  Created wheel for pybedtools: filename=pybedtools-0.9.0-cp310-cp310-linux_x86_64.whl size=14248332 sh

In [3]:
import numpy as np
import pandas as pd
import pybedtools
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from captum.attr import visualization as viz
from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer
import torch
import matplotlib.pyplot as plt

In [4]:
from transformers import AutoTokenizer, BigBirdForSequenceClassification, AutoModelForSequenceClassification, AutoModel

tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/SE_annotation_project/bulk/results_bert/bigbird_base')
model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/SE_annotation_project/bulk/results_bert/bigbird_base')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
model.zero_grad()

In [5]:
def predict(inputs):
    return model(inputs)[0]

def construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id):

    text_ids = tokenizer.encode(text, add_special_tokens=False)
    # construct input token ids
    input_ids = [cls_token_id] + text_ids + [sep_token_id]
    # construct reference token ids
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(text_ids) + [sep_token_id]

    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device), len(text_ids)

def construct_input_ref_token_type_pair(input_ids, sep_ind=0):
    seq_len = input_ids.size(1)
    token_type_ids = torch.tensor([[0 if i <= sep_ind else 1 for i in range(seq_len)]], device=device)
    ref_token_type_ids = torch.zeros_like(token_type_ids, device=device)# * -1
    return token_type_ids, ref_token_type_ids

def construct_input_ref_pos_id_pair(input_ids):
    seq_length = input_ids.size(1)
    position_ids = torch.arange(seq_length, dtype=torch.long, device=device)
    # we could potentially also use random permutation with `torch.randperm(seq_length, device=device)`
    ref_position_ids = torch.zeros(seq_length, dtype=torch.long, device=device)

    position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
    ref_position_ids = ref_position_ids.unsqueeze(0).expand_as(input_ids)
    return position_ids, ref_position_ids

def construct_attention_mask(input_ids):
    return torch.ones_like(input_ids)

def custom_forward(inputs):
    preds = predict(inputs)
    return torch.softmax(preds, dim = 1)[0][1].unsqueeze(-1)

def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    return attributions

In [6]:
def dataframe_creation(attributions_sum, all_tokens, start, chrom):
  short_a_s = attributions_sum[0:len(all_tokens)]
  df = {'tokens':all_tokens, 'score':short_a_s}
  df = pd.DataFrame.from_dict(df)
  df = df[1:len(df)-1]
  starts = [start]
  ends = []
  l=list(df.tokens)
  for i in range(len(l))[1:]:
    starts.append(starts[i-1]+len(l[i-1]))
    ends.append(starts[i])
  ends.append(starts[-1]+len(l[-1]))
  df['chromStart'] = starts
  df['chromEnd']= ends
  df['chrom'] = [chrom]*len(df)
  return df

def bed_file_creation(df, filename):
  df_pos = df[df['score']>=0]
  df_neg = df[df['score']<0]
  mult = 1000/max(max(df_pos['score']), abs(min(df_neg['score'])))
  df_neg['score']= df_neg['score']*mult*(-1)
  df_pos['score']= df_pos['score']*mult
  pos_data = pd.DataFrame()
  pos_data['chrom'] = df_pos['chrom']
  pos_data['chromStart'] = df_pos['chromStart']
  pos_data['chromEnd'] = df_pos['chromEnd']
  pos_data['name'] = df_pos['tokens']
  pos_data['score']  = df_pos['score'].astype(int)
  pos = pybedtools.BedTool.from_dataframe(pos_data)
  pos.saveas(filename+'_pos.bed')
  neg_data = pd.DataFrame()
  neg_data['chrom'] = df_neg['chrom']
  neg_data['chromStart'] = df_neg['chromStart']
  neg_data['chromEnd'] = df_neg['chromEnd']
  neg_data['name'] = df_neg['tokens']
  neg_data['score']  = df_neg['score'].astype(int)
  neg = pybedtools.BedTool.from_dataframe(neg_data)
  neg.saveas(filename + '_neg.bed')

#SE_67479

The text is SE_67479 from SEdb v2.0, 11.06.2023 18:00

In [ ]:
text = 'AAAGGTTTGGAGAGGGAGGTCCTAGTGAGGCTGAACAGAGGTAAAATAAACAGAAAGTTCCAGGCTCTCCTTGCTTCTACTTAAGAAAAATCAGAGCAGTAAATCCCAAACCAGTTCTGCCCACAACTCAAATGCAGATGAGCATCCCTTGCTCCATCTATGCAGGAAAGCCATAGTGTGGGGCTGTAGGACTATGGAGTCAGCCTCTGCAAGCAACAAGGCTCAGAGGCTGAGGGGCTTCAGGCCCCAGCAAAACCCAGTGATCCTGTTGTCAAATCTGAATGTCATCATCTGCCTCCAGGGCGACCAGCTCTGGATGGGGGTGTCAGGGTCAGGATCCTGTTCCATGTGGCAAGAGCTTGGGGAAGCTGCTGCAGGCCTACTGATGACCTTCTTTCCTGGCCCAGACACTCTTAAAAACTTCCTGGCGAGGGAGGCAACACACCTTATTGTGCAGCCCTCATCGAACGCATCAGCTGCACTAAGTGCTGAAGAAAAGTTCCCATAATAACCATGTGTGATAATAAAAAGATAAAGAGGCAAATAGGTATGTAGGTCAGCTCAAAAATAGCCACCCACCCGCCTGGCCTGGGTAGCCCTTTGTGCCAAGTGGGTAGAAAGGTTTTCCTTGCTTCCCTTCCCAAGCTGATGCCCCCTGCACTTTTCAAGGTTGCCTTAAATACAACTCCTTCCTGCACTAGCTGGGGTAGCCAGGGCAGTAGGGACTGCAGCCCTACAGGGGCAGCCCTGCGCTCCCTCCACCAAATCAATCCTGGGGTCAGAGGGCACAAAGGCTTGGCCTCTCTGACGTGAGTTCTCGGGGCTAAGTCAGGGTTTAGGGTAGTGAGTCACAGACATTCTTTCCCCCATCCCTTAGGGTTTGCGCACACCCAGGAAGCCTTCCCCTCCCCAACAGAGACCGCTTTGTTAGCACGACCCACCCGGTTGGAGAAGAGTTACAGCCTCAGGGGTGTGAGGGCCTCCTCGGCCTCTCCCTCCCTCCCCCTCTCTCCTCTCTAGGGCTCTCACCAGAAAGGCGCCAACCCTGCGCCCTGCCCCCACCCAGCCGCCGGGGCCCCAAATGCTGAGCTCAGCATCTCTGTGGCCTGGCTGCCCTCCTCCCGGGCAGTGGGAGGAGCCTCCTTCCTCCCGAGGGTGGGCGGGGCCAGGGCCAGCCCCGCCCTCCTCCCCGTTCCCCCTCTCCCTCCATCCCCCTCCCGTTCCTCAGTCGCACGTTCTCCAAGTCACGTCCCCCGCTTCCCCTAGTAGGGGGTCGACGGAGATTGGACCCTTCCCCCAGTCAGCTGCTCTGCATCTGTCACCTCGGCGGCAGCGCTCCCTTGAGGTCGCACCTCCGCTGTGGAGGGAGGCGAGAGCGAGCTGAGTGGGGGGTCTTCCCTTCCCCCATCCCGCTGGCCCCAGACAAGCCACCGCAGACCGCGGCGAAATCAGTTAAGCTCTTGCGTGCACCAGTTCCCGGAGCAGAGTCTGAAAGTTTTTGTGTTTCTGTGCGGCTACTTTTAAGTATAAAAAATTCAGCAGTGGCTCCTGGGTGCCTATGTCAGCGTCTGGCCCTTTGTGGATTACGTACGGGACAACTGGGGTGGGGAAGGGCCCCTCTTCACAGGCAGCAATTCCCAGAGTGGCCTTCCTATGGGACCCCAGAGCTCCCAGGGGGAGGATCCCAGAGGGGGATATGGTAACAGAATCACTGTCACTTCTGGGATTGAGGGTGGCAGCTAAAGGGAAACCCCTCGAGTGTCAGTCGGACTTGCCATTTGAGTCAGCACTGCTAGGTGCTGCTGAATTCTGCAGGGGGAGGAAGAACCTGATCAGATAAAACAAGGGCTGACCTAGAGAAAGTCTCTTGGGTCAACATATTCTTAGCGATAACAGAAGATCACTCCAACTGTTCTACACCCCACCCTACAACATTGACCCCCCGCTTTCCAGCATTTACAGTCCCTTTTAGTTTACAACTAAGAGTATGGGTTGCCTGGGATCAGGAAATGAGCCTAGATGTAGAGTCCTGGCTCCAGCCTTGCTGACACTATGTGACTCACTGCACCTATCTGAATCTCAGTTTTCTCACCTGTGCAGTGACATTGATGAGGTCTGTCCTGCCTACAAACTCTACACACATATATTAGTTGTTGTTTTTTAATTTTTAGTAGAGATGGGGTTCTCACAACTTGCCCCAGGCTGGTTTGAACTCCTGGATTCAAGCAATCCTCCCATCTCCACCTCCCAAATGCTGGGATTACAGGTGTGAGCCAGTGTACCAGGCCAGTATTATGCTGGGATTACAGGTGTGAGCCACCATACCAGGCCTTTATTTTTGTTTTGTTGGTTTGTTTCGTTTTTGTTTGAGATAGCGTCTCACTCTATCACCCAGGCTGGAGTGCAGTGGAACGATCTTGGCTCACCGCAACCTCCGCTTCCCGGGTTCAAGTGATTCTCCTGCCTCAGCCTCCCAGGTAGCTGGGATTACAGGTGCATGCCACCAAACCTGGGTAAATTTTGTATTTTTAGTAGAGACAGGGTTTCACTATGTTGGCCATGCTGGTCTCAAACTCCTGGCCTCAAGTGATCCACTCGCCTCAGCTTCCCAAATCATGTGCTGGAATTACAGGTGTGAGCCACTGCCCAGCCTAAGTCTTTTTTTTTTTTTTTTTTTTTTTTGAGACAGAGTCTTGCTTTTTGACCCAGGCTGGAGTGCAGTGGCACAATCTCAGCTCACTGTAACCTCAGCCCCCTGGGCTCAAGTGATTTTCCCACCTCAGCCTCCCAAGAAGCTGGGAATACAGATACACTCTACCATGTCCAGCTAACTTTTTGTCTCTTTAGTAGAGATGGGATTTCACCATGTTGGCCAGGCTGGTCTTGACCTCCTTGACTCAAGTGATCTGCCCACCTTGGCCTCCCAAAGTGCTGAGATTACAGGCATGAGCCACCATGCCCAGCCTCCAGTTGTTATTTTTATTAACAAAGTTATGAAAATCCTTTAGAACATGTGGAAAAAGACAAAAAGTTATAAGGCTGGGTGCAGTGCCTCACACCTGTAATCCCAGCACTTTGGGAGGCCGAGGTGAGCAGATCGCCTGAGGTCAGGAGTTCAAGACCAGCCTGGCCAACATGGTGAAACCCTGTCTTTACTAAAAATACAAATATTAGCCAGGCATGGTGGCGGGCACCTGTAATCCCAGCTACTTAGGAGGCGGAGGCAGGAGAATCGCTTAAATCCAGGAGACAGGTTGCAGTGAGCCACGATCGTGTCCAGCCTGGGCAACACAAGACTCCATCTCAAAATTAAAAAAAAAAAAAGTTATGAAAGCCCTAAAAAATATGGAAGTGTGTTTTTTTGTTTGTTTTGGTTTTTTTGAGACGGAATTTCACACTGTTGCCCAGGCTGGAGTCTAGTGGCGCGATCTAGGCTCACTGCAACCTCTACCTCCCGGGTTCAAGCGATTCTCCTGCCTCAGCCTCCCGAGTAGCTGGGATTACCGGCGCCCGCCAGCACGCACGGCTAACTTTTTCTATTTTTAGTAGAGATGGGGTTTTACTATTTTGGCCAGGCTGGTCTCAAACTCCTGACCTCATGATCCGCCTGCCTTGGCCTCCCAAAGTGCTGGGATTACAGGCGTGAGCCACCGTGCCAGGCCTGTGCAAGTGTTTTTAGAATCCTGAAGCGCTCCATAAGGGTCAATGATCTTTGTGAGGAGGCTGGTGCGGTGGTTCACGCCTGTAATCCCAGCACTTTGGGAGGCTGAAGGGGGCAGATCACCTGAGGTCAAGAGTTTGAGACCAGCCTGGCCAACGTAGTGAAACCCTGTCTCTGTTGAAAATACAAAAATAAACCGGGCGTGGTAGTGCATTCCTGTAATCTCAGCTACTCGGGAGGCTGAGGCAGGTGAATCCCTTGAAGCCGGGAGGCAGAGGTTGCAGTAAGCTGAGATTGCACCACTTGTACTCCAGCCTGGGTGACAGAGTGAGACTCCACCTCAAAAAAAAAAAAAGATCTTTGTGAAGAGATGGCAAATTGCCAGGTGGGCTGGCAGCTTCCATTTCCTGCCAGCCCAGGTGTTCCAGAGAAAGGCTCAGCATGCAGACCTGGGCAGCTGCCTTTTGCCTAGACTTGAGTTTCCCAAGCCTTTGACTATTAAGGCCACTGGAGATAAAATGCACTTCTGAGTGAGGTTAGAGGCATCGCAGAAGGGTGATGAAGGAGACCAAAGCCAGGGAGAAGAAAGTTCCAGAGGATTGAGTGGGGCCTAACTGCCAGGAGAGAGGGAGATCCTGGAACCTCCTAAGGCTCCCTGCCCTGTCAGTTCCTTGTCCATAAAGTGGGGGAGGTTGGGGGCTGGAAGGAGTACTCTCTTAGGACCTGCTTATCTTAACATTAAAAGATTTTAGCCGGGCATGGTAGCTCATGCCTGTAATCCCAGCACTTTGGGAGGCTGAGGCGGGCAGATCACCTGAGGTTGGGAGTTTGAGACCAGGCTGACCAACATGGAGAAACCCCATCTCTACTAAAAATACAAAAAAGTTAGCCAGGTGTGGTGGCACATGCCTGTAATCCCAGCTACTCAGGAGGCTGAGGCAGGAGAATCGCTTGAACCTGGGAGGCGGAGGTTGCGGTGAGCCAAGATTGCGCCATTGCACTCCAGCCTGGGCGACAAGAGCGAAACTCCATCTCAAAAGAAAAGAAAAAAAGAAAGATTTTAAGGTTCATTCTACTGACAGAAGAATCCTGACAGTGAGAAGGCTGACCCTTCTCTCTGCTGATCTAGCTCTACTCACCCCAGCCCTTTGGGAAACGCAGAGGCTAGACAGGTCCTCCTGGTCTCCTTTCTTCTCTAGAAGGAAAAACAACAACAATAGAGTGAGCTCATTTTGCTGTCCAGACAAATGATGAGCCTGGGTTTGGGCTTTCCCAACTGTGGCAGGTCCTGATCATCTCTGTCATCCCCTGTCATATACTCCCTAGGCTGGTGGAAAAACCCCCATCCCTGCTTCCCCCTTTGTTATATGGATATCAACAATATTTCCATATTGTAAACCAATTGGAAGCCCTCAAAAGGCAAAAGCACTGAAATGGCTGAAATGCCTATTTCTACCTTCTCTGGCAGGCCTCCTGCAGAGAGTCCTATGGGCTAGGACTGGGGTTTGGGGGAAAGCAGCTCCATGGAATGTATTATATCTTTCTCCTTTGCTATGATCCTTGTAGAGGTTGAAGAGGTCTTTGAGATTCTCTAGTTCCGTGTTTTCAAAACTGAGGTATGAAGGCCGGGCTTGGTTACTCACGCTTGTAATCCCAGCACTTTGGGAGGCCGAGGCAGGCAGATTACTTGAGGCTAGGAGATAGAGACCAGCCTGGCCAACATGGCAAAACCCCGTCTCTACTAAAAATAAAAAAATTAGCCAGGCGTGGTGGCGCATGCCTGTAATCCCAGCTACCCGTGAGGCTGCGGCAGGAGAATCTCTTGAACCTGGGAGGCAGAGGTTGCAGTGAGCTGAGATATCGGCACTGCACTCCAGCCAGGAGGACAGAGCGAGACTCTGTCTCAAAAAAGAAAAAAAAAGTTTTTTTCTTTTTTTGGAGGGAGAAATTAAGTAATTTAACTAGTAAGTAATCTAAACATTTGCTACTGAAATTAACACAGATAAAATATATAATAAAATGCAAAATTGTGTTATTTGTAATAAAAATTCTGGGCTTCAAAGACATGGTATGCTTCCTGGGGGCTGCTTCTGGGCCACAGCCCAGACTCCGATCCATCCTTAGTGGCCAAACAGCCTGCAACTAAGGTCTGCTGACTCCTATCCTCTCACTCCACAGGGCCTGAGCTAGCCGAGGTTTAAGAGACTCACTAGAGAGAGGGGGTTGCCTAGCCCTCCCTGCACTTCATTGCTTTCTCTCCTCATAAGGATTCAGAACCTAGGATTGGAGAGGGATCTTAGTAATGGCTGGCAGAAACCTGGCCTCTAAGTGGATGGGATTCCTTACTGACAGTCAGTGGGACAGGTGAGGAGAAAAGAGACAGTGCCGGAGTCAGAAGGACGAGGTTCCAGGCCTGGCTGGGTCCCTAGGACAACTCATTTGACCACGCGGGCTGGAGACGGGTGGGCTGCATGAAGTCTGGTAGGCTGTCTGTGCCTTTGCTACTGCTGGGATACAGAGGTCTGCCAGAATCCTCAGGCTACTCTCTCAGCTCTCAAAACCCTTATCCCTTGGCATGGGTGGTTTATCAACCTAGCCCTTTGTGGGGTTCCCACTGCCCTGCCCTCTCCCCAGAGGAAATGACTTCTTGGGAAACCTAGGACCCCAGGCAGAACCAGGCAGAACTGGTTGTCTGTGAGCTCACCTTTGTCACGTGTGAGCCCTGTCTACACAGCTGCAAGCTCTCACCCACCCCTAACCCAACCAGGCCTATCTTTCTTTCCTAGAGGTCAATAAGCACCCCAACTAAAGATAAGCATCAGAATCGCCTGGAGAATTTTGCTCCTAAGGACCCCACCCAGACCTACCCAATCAGAATGGGGGTTGAGGAGATGAGAATCTCGGTTTTACAAGCTCCTCAGCTGATTCTAATTTTTTTTTTTTTGAGATGGAGTCACACTCTGTTGCCCAGGCTGGAGTGCAATGGCGTGATCTCAGCTCACTGCAATATCTGCTTCCTGGGTTCAAGCGATTCTCCTGCCTCAGCCTCCTGAATAGCTGGGGCTACAGGGGTGTCCCACCACACCCAGCTAATTTTTGTATTTTTAGTAGAGATCAGGTTTCACCATGTTGGCCAGGCTGGTCTTGAACTCCTGACCTCGTGATCCGACCGCCTTGGCCTCCCAAAGTGCTGGGATTATAGGCATGAGCCACCACACCCAGCCTGATTCTAATTATTAATCAGACTTGAAATGAAAGCCACTGGTTCACGGAGGGATTACGGGACTCTCACTTTGAGCCTCAATGTCCACATCTGTAGAATGGCACTATATTTGTTCCTTGGGGCATACTTGGACAACAGATTTTTTTTTTTTTTTTTTGAGATGGAGTTTCACTCTTGTTGCCCAGGCTCGAGTGCAATGGCACAATCTCGGCTCACCACAACCTCTGCCTCCCAGGTTCAAGCGATTCTCCTGCCTCAGCCTCCCTAGTAGCTGGGATTACAGGCATGTGCCACCACGCCCGGCTAATTTTCTATTTTTAGTAGAGATGGGGGTTTCTCCATGTTGGTCAGGCTGGTCTCGAACTCCCGACCTCAGGTGATCTGCCCGCCTCAGCCTCCCAAAGTGCTGGGATTACAGGCATGAGCCACCGCGCCCGGTCTGGACAACAGATTTTTAAAAGCTACTTTGAAAGACTGCTCAATACCGTCCAGGCCCGGTGGCCCATGCCTGTAATCCTAGCACTTTGGGAGGCTGAGGCAGGCGGATCACTTGATGTTAGGAGTTCAAGACCAGCCTGGCCAACATGGTGAAACCCCGTCTCTACTAAAAATACAAAAATTAGCCAGGCGTGGTGGCACGTGCCTGCAGTGCCAGCTACTCAGGAGGCTGAGGCAGGAGAATCACTTGAACCTGGGAGGCAGAGGTTGCAGTGAGCAGAGATCGTGCCACTGCATTCTAATCTGGGTGACAGAGTGAGACTCCATCTCAAAAAAAAAAAAAAAAAGACTGCTCAATACATTGTATTGTTAGTATTTCCATTTCAAAAAGCAGCCTTATTTGCAAAGACTCCTCACTGAGTTTTTAACTCTCCCTCCAGTAGGATAGACAAAATCCTCCTTGAGGCCCTCACTTCAGCTTTTCACAAGCCTCCTAAACCAGGTGCATTTATAATCGTGCTATTTTGACCAAGCTGTTTGTTGTTTTTCAAGAAAGCCACAGAATCCTGGGAACGCTTCTGCTGGCTTCTCATCTCCACCTCCTCCCAATGCTCCACCTTCTCCTTCATGCCTCGACCCACGGGCTCCTCCACCAGCTACTTCCACCAGCCGCAATGAGAAATTTCCAAAGGGCTGCAGAGGCGCCCCAAGCAGAGGCATGCGAGCGAGCTGGCTGGCCTTCAGGGGTTTCCATCCTGAATTCTGGGGTTCTTGGAATCATACCCCCACCCCAAGATCTGGTTCAACTTCCCAAACCCGACCAGGGTGACTTGCCAGACCGAGGCCCTGGGAACCGGTGCTCCAGACGGAACCGCCTGAACGGGGAGGGGCAGGTGGCCGGGAGTGAGTGCTCATTGACTTTAAATTGCAGGTTGCGGCATTGCGCTCTATTCTGATGGAGTGCAGTCCTGGAAGAGGCAGGGTTTATGCACGGTCAGGGCCCTGGCCACAGACACTGGGACATGGGGCCTGATCACCAACTCCAGAGGCCTGAGTCACCTGGGACATGACACCAGCCCACTCCTGCTCAGAGGTCGGCCAGTCTCCCCACCAGGGTTGTCTTGCGGCTGCCGTGTTGCACATTGTGTCTATGTGATGAGTGCTCCCTGGAGCACTACTCTCCAGAAGACACATGTTCATGCAGACTCTAATGGTAGCAGCATCCTCTGGAGTTGTGCAAAGAGGCAGCCCTGCTTCTTCTTCTTTTTTTTTTAATTAAAAAAAAATAGGCCAGGTGTGGTGGCTCACACCTGTAATCCCAGCACTTTGGGAGGCCAAGGCGGGTGGATCATGAGGTCAGGAGATCGAGACCATCCTGGCCAAAATGGTGAAACCCCATCTCTACTAAAATATGAAAAAATTATCTGGGCATGGTGGTGTGCACCTGTAGTCCCAGCTACTTGGAAGGCGGAGGTTGCAGTGAGCTGAGATCACTCCAGCTTGGCAACAGAACAAGACTCCGTCTCAAAAAAAAAAAATAGAGATGGGGGTCTCACTATGTTGGCCAGGTTGGTCTTGAACTCTTGGCTTCAGTCAGTCCTCCCACATTGTCCTCCCAAAGTGCTAGGATTACAGGCATGAGCCACCACACCCAGCCCTACTTCTACCAACAAGGGAACACTTCAAAGCATTTTGAGGGGAGTGACTGCTTCATGGGTAAAGGACCTCCATGGGAGTTAGGAAAGGTGTTTTGGAACCAGATAGAGGTAGTGGTTGCACAACATTGTGAGTGTACTAAATGCCACTGAACTAACTGTTCACTTTGAAATGGTTAATTTCTCTCCCTCCCTTCCTTCCTTCCTTCCTTCCTTCCTTCCTTCCTTCCCTCCTTCCTTCCTTCTTTCCTTCCTTCCCTCCTTCCTTCCCTCCTTCCTTCCTGTAATCCCAGCTACTCAGGAGGCTGAGACCTGGGCAACAAGAGTGAAACTCTGTTTCCAAAAAAAAAAAAAAAAGGCCAGGTGCGGTGGCTCACGCCTATAATCCCAACACTTTGGGAGGCCTAGGCGGGTGGATCACCTGAGGTTGGGAGTTCAAGACCAGCCTGACCAACATGGAGAAACCCTGTCTCTACTCAAAATACAAAATAAGCTGGGCATGGTGGCGCATGTCTGTAATCCCAGCTACTGAGGAGGCTGAGGCAGGAGAATCACTTGAACCCAGGAAGCAGAGGTTGCAGTGAGCCGAGATCGTGCCATTGCACTCCAGCCTGGGCAACAAGAGCGAAACAGCGAAACTCCGTCTCAAAAAAAAAAAAAAAAAAAAATTCCCTACAACAGATCAGCCCTCTGCAAGATTTCACGTCTCTCCCCAGTTTCCTGGGCTAGGTCGCTGACTATATTCCAGAGGAAGAACTGGATTGGCTTAGAAGCCCCATTCCCAGACCCCAGCTGAGGAGGAAGGGAAATGGTCAGATCTGTCCAAGGCTCTTGGCAGCACTTGCCATAAAGAGGATGGAAAGGTAGCATCAGGGTCCTCTTGAAGGCACCCCATAGGCCCCACTAGATCCTGGGAATCTAGATGCGGGTGTCTGCAATTCTGGAAAAAGCCATGTGCTTTTTGTTTTTGTCTCACTATGTCACCCAAGCTGTAGTGCAGTGGTACCATCATAGCTCACTGTCCCTGGCCAGGCTCCAGCGATTCTCCTACCTCAGCTTCCTGAGTACCTGGGACTAAAGATGTGTGCTACCATGCCCAGTTAATTTTTATTATTTATTTATTTATTTATTTATTTATTTGAGACAGAGTCTCGCTCTGTCACCCAGGCTCGAGTGCAGTGGCAAGATCACGGCTCACTGCAACCTCCGCCTCCCGGGTTCAAGCAATTCTCCTGCCTCAGCCTCCCAACTAGCTGGGACTACAGGCACGCGACACCACGCCCAGCTAATTTTTGTACTTTTAGTAGAGATGGGGTTTCACCATGTTGGCCAGGATCGTCTGGATCTCTTGACCTCGTGATCCACCTGCCTCAGCCTCCCAAAACGCTGGCGTGAACCACTGCATCTGGCCAATTTTCTGTATTTTAGTAGAGACGGGGTTTCACCATGTTGCCCAGGGTGGTCTCAAACTCCTGAGCTCAGGTGATCTGCCCACCTCAGCCTCCCAAAGTGCTGGGGTTACAGGCATCAGCCACCATGCCTGGCTAATTTTTAAATTTTTTGTAGACACGGGGTCTCACTATGTTGCCTAGGCTGGTCCTAAACTCCTGGCTTCAAGTAATCCTCTCACCTCCAAAGTGTGTGGGTTAAAAGTGTGAGCTGCCACAGCCAGAGAAGCCACGTTCTTGCCTCATCTAGCCATTCATTCATCCATCCATTCATTCATGTATTTATCCAAGGAACATTTGCTGAGCCTTTTCTATGTACTGGGCACAGTGCTGGCTACTTGAAGTTAGGATGTGTAGAACGCCATCCTCAAGTAGCTTACTGTCTAGTGGGAAGACCAAAGTGGAAAGAATTATTAACAACAAAGATATGCACTGGCTGGAAAGATGCCCTGAGGAAGGCACCTAAGGGATGGAGTGTCGGGATGAGGGTGCTAGGGGAGGCTTCCTGGAGGAAGGGATGCCAGAGTTGGGTGGCCAAGGCTGCTGGTGGAAGCATGTCTAGGCAAGCCCACCTTGGCCTGAGGGGAGGAGGAGCGGGTGGGTGGACACCAAATGCAGGGTTTCTAGGTTTTGTGAGAGCTCCAATGAGGTCTGGTTCAGAGGCCAGGAGATGGAGCGTCCTCCTGACCAGAAGGGCCCCATTTGCAGAGCTGTGGAGTGAGGGGGTGTTCGGTGTAGGGAGGGTGGTTACAAAGTCTTTCCAGCTCCAGCCAGATGGAAGGGAATCTACCCTCCACCTCAGGACAGGATGGACCAAGGCCCTGCACTCTGGCTCCTGGGAGAGGCTAGCAGTTAAGAACCCCCACTTTACCCTCCTCCCGACCTTTGAGATTCTCCTGGCATACAGGCCTTTTCTCTATCAGGGGTTTTAGCAGTTGTCACAGGGGAAGACACTGACCACTCCCAACTCCCCCAGGCCTCTGCCAGGCCTCCTTGCAGGAGACTTTACCCTGGAACCCACTCCCTCCTACTTTAGGAAACACTTCCAACAATAGGACTGCCTTCTTTTTTTTTTTTAATCGCTTTTTAGAAACACAATGAGACAATAACACTAATAGATTTCTTTTGAAAAGAAACAAAATTACTCAGTGCGCATTAACCAGCCCTTCCCATTTGAGACAAGTTGTTTTATTAACAGAAGGAGACAGTGAATGAAAAAAAGGTGCTTGGAACACTGTTGGGCAGTCTTGATGGTGTTTGTTGAATGAATAAATGATTCCTAATAACTGACCTTTCCAAACCGGACCCGGATTCTTTTCCTTATGAATATCTACTTAAAAGGAAAAAAAAAAAAAGCCTGTGCTTTCTTTAGGTCAGTGTTCCACATTTTCCTGAATTTTTTTTCTGTCCAAAAAAAAAAAAAAAAAGGCCAGGCGTGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCCAGGCTGGTGGATCACCTGAGTTCAGGAATTTGAGACCAGCTTGGCCAACATGGTGAAACCGTCTCTCTACTAAAAATACAAAAATTAGGCCGGGCGCGGTGGCTCACGCCTGTAATCCTAGCACTTTGGGAGGCCGAGGCAGGCGGATCACGAGGTCAGGAGATCGAGACCATCCTGGCTAACACGGTGAAACCCTCTCTCTATTAAAAATACAAAAGATTAGCCGGACGTGGTGGTGGGCACCTGCAGTTCCAGCTTCTCGGGAGGCTGAGGTAGGAGAATGGTGTGAACCCGGGAGGCGGAGCTTGCAGTGAGCCAAGATCGCACCACTGCACTCCAGCCTGGGTGAAAGAGTGAGACCCCATCTCAAAAAAAAAAAAAAAAAGAAGAAGAAGAAGAAAAAACATCGTAACTCTTGTTTCCTTAGGGTGTTTAGGTGTCCCGCCCAAATCAGCCTGGAGGAAATGTGAATGTTCCTGATGGATTAATTCAGATCTTCTCTCTGGCCAGCAGGAAGACACACCCATGCAGGTCACCTCTGTAAGAGTGATTTCAAGAGTATGTCATCTACTCCAGCCCTCCAAGAACAACTAGCCGCCTTGATCCAGTGTCTTTTATTATTTTTAATTTTTATATCCCCATTGTTAAGCAATCTGGTGTCTTTAAGGCTCCAACAAGCGACAGAGCTTGAAGAATGGGAAGAGAAGGCCTTGAAAGTCCCAGTGGCTCACAAACTATTCCATTCTGCAGCTGTCTTTAAGATGTCTCCTGGATCCCCTGATTTCTTTTTTTAATCCATAAATTTGTCTCCATAAGTGGGTCTCCCACCTGTCAAATGAGAGTAGGGTGCAGAGCTTGGATTCTGGTTCTTTCAGGTTCTAAGTCTTGGCCCAAGTGTCAGCTTTTTCTTTTTTCTAAGTTGGGGTGGGGATGGGGGTTGCGGAGGGCAGTGGCCTGACATCTCCTGCAAGTGTCATTTGTCACGCGAAAGCAAGCCTGACCCGAGGCAAGGCCCGCCTATCTGGTTAGGTTCCTTGCCCGCCCCCGGGGCCCTGGGCATCTGATGTAAGTTTGGCTCCCTCTTCCATCAGGGTCCTCCTGTCCCAGACTGATAAACCTTTAGAGCAATAAAGGTCGAGTCCGTCTCAACCTCCACCCACCCTCAGCAACCACTGTTAGCCTCTGAGCACTGCTGCCAGGCCAGATTTCCTGTGGCCCCGCCTGAATGATGAAACACGGGATGGCCATTGCACTCCCTGGCTTTTCCAGAGATCTCGGTCCTCGGTCCTGATGCAACCGTCTGGATCTTTTACACAAAGCGCTAGGCGAGCGCTGGGTGCTAGGACCCGGGGGGCGCCCGGGCAGGGAGGTCCGGCTGCGCTGCCTGCAGTCCGCGGGGAGGGCGGATGGGTCTGTGAGTCCGACTGCTGTGAATCCTCGCTTCCATCCCTGCACTGGGCGCTATGAACCCGGGCCCATCCACACTCCTCTAACCACTCTCCTCAAGCCCTCTCCTTGGGAGCAGTTGGAAGTGGAAGCAGATAGGGTTTGGGAGGAGGCAGGGATCCCCATTAACCTATGTCCTGGGCTGTAGAGTAACAGCCAGGATGGGCAGGTGGGGCCTTGGGGAAGGGGCAGCATGGTGCCCGGAGCTTCCCAGTCTGTGGACCCCACCCCCATATCCCGAGCCTTTGCAAAGCAGGGATAGAGGGGGTAAGGGCGCGAGGGCAGGCCGGGCTAACGTCTGCTCCTGCGCCGCCGTCCCACGGGCATCCCCCCCGTGTGTGCCCAGTTGTTCCCGAAGCTTTAGGATCCAAGACGCTGGGGGCAACCATTTTCCTTGCCCGCCGCCCCCTCACGCTTCCCTGCCTCTCCTCCTAGCCTGGTCACGGGCTGTCCCCTCCTCCAGCAATGCAACCCTATAATAAACAAGTCTTTCCTTGATCCTCCCCTGCCGCGAGCGCCCTCGGGGACCTTGGCAGCTGCAGCCGCCGCGGATCCTTTCCAGAAAGGGGGCGTGGCGGTGGGTCGGGGTTCGACTGCGAGCTTTCTGGGGCTCAATGGAGGCGGGGCCCGGCCGCTGTCACCGGGCAGGAGAGAACGTTGCTTACGTGCGCCCGGAGTCCATTGGCCAAGGCGGGCCACACTCCCGGGTCTGGATTGGGTCGTGGCGCAGAGAAGGCGTGGCCTCGCCGCGCTAGTCCTTATAGGCTGCTCCGCGCTGGTGCTAGGGCGCAGCAGGCCAAGGGGGAGGTGCGAGCGTGGACCTGGGACGGGTCTGGGCGGCTCTCGGTGGTTGGCACGGGTTCGCACACCCATTCAAGCGGCAGGACGCACTTGTCTTAGCGTTCTCGCTGACCGCGCTAGCTGGTGAGTGTCCCTTCTGTGTGTGGGTCCTAGAGCTCGCGGTCTGGTCTGGTCTGGTCCCCAGACTGACGCCTGGTCGGTCCCCCTCTTGTCTTACAGCGGCTTCTACGCTCCGGCACTCTGAGTTCATCAGCAAACGCCCTGGCGTCTGTCCTCACCATGCCTAGCCTTTGGGACCGCTTCTCGTCGTCGTCCACCTCCTCTTCGCCCTCGTCCTTGCCCCGAACTCCCACCCCAGATCGGCCGCCGCGCTCAGCCTGGGGGTCGGCGACCCGGGAGGAGGGGTTTGACCGCTCCACGAGCCTGGAGAGCTCGGACTGCGAGTCCCTGGACAGCAGCAACAGTGGCTTCGGGCCGGAGGAAGGTGAGCGGTGGGCGGGTGCCGACGCGACTCGAGGGGCCGGGAAGGTGGGGAAGCCCCGGTGCTGGAAGGGGTCAGAGCCGCCTTGGCTTCCTATCCCATCGGGACCCAGATTGCTTGGGGGCAGGGGGAATTGGAGTATAAGGTGAGTGAGGCGGAAACTGAGGCACGGAGTGGGAAGGAGCGTTGGTTTCTTAAGGAAACAGCACCTCCCCCGCCTGTGCGTTTCGTTTTGAAGCCGCTCTAATACCCCTTCCTGTGTGCTCTCCTTTCCAGACACGGCTTACCTGGATGGGGTGTCGTTGCCCGACTTCGAGCTGCTCAGTGACCCTGAGGATGAACACTTGTGTGCCAACCTGATGCAGCTGCTGCAGGAGAGCCTGGCCCAGGCGCGGCTGGGCTCTCGACGCCCTGCGCGCCTGCTGATGCCTAGCCAGTTGGTAAGCCAGGTGGGCAAAGAACTACTGCGCCTGGCCTACAGCGAGCCGTGCGGCCTGCGGGGGGCGCTGCTGGACGTCTGCGTGGAGCAGGGCAAGAGCTGCCACAGCGTGGGCCAGCTGGCACTCGACCCCAGCCTGGTGCCCACCTTCCAGCTGACCCTCGTGCTGCGCCTGGACTCACGACTCTGGCCCAAGATCCAGGGGCTGTTTAGCTCCGCCAACTCTCCCTTCCTCCCTGGCTTCAGCCAGTCCCTGACGCTGAGCACTGGCTTCCGAGTCATCAAGAAGAAGCTGTACAGCTCGGAACAGCTGCTCATTGAGGAGTGTTGAACTTCAACCTGAGGGGGCCGACAGTGCCCTCCAAGACAGAGACGACTGAACTTTTGGGGTGGAGACTAGAGGCAGGAGCTGAGGGACTGATTCCTGTGGTTGGAAAACTGAGGCAGCCACCTAAGGTGGAGGTGGGGGAATAGTGTTTCCCAGGAAGCTCATTGAGTTGTGTGCGGGTGGCTGTGCATTGGGGACACATACCCCTCAGTACTGTAGCATGAAACAAAGGCTTAGGGGCCAACAAGGCTTCCAGCTGGATGTGTGTGTAGCATGTACCTTATTATTTTTGTTACTGACAGTTAACAGTGGTGTGACATCCAGAGAGCAGCTGGGCTGCTCCCGCCCCAGCCCGGCCCAGGGTGAAGGAAGAGGCACGTGCTCCTCAGAGCAGCCGGAGGGAGGGGGGAGGTCGGAGGTCGTGGAGGTGGTTTGTGTATCTTACTGGTCTGAAGGGACCAAGTGTGTTTGTTGTTTGTTTTGTATCTTGTTTTTCTGATCGGAGCATCACTACTGACCTGTTGTAGGCAGCTATCTTACAGACGCATGAATGTAAGAGTAGGAAGGGGTGGGTGTCAGGGATCACTTGGGATCTTTGACACTTGAAAAATTACACCTGGCAGCTGCGTTTAAGCCTTCCCCCATCGTGTACTGCAGAGTTGAGCTGGCAGGGGAGGGGCTGAGAGGGTGGGGGCTGGAACCCCTCCCCGGGAGGAGTGCCATCTGGGTCTTCCATCTAGAACTGTTTACATGAAGATAAGATACTCACTGTTCATGAATACACTTGATGTTCAAGTATTAAGACCTATGCAATATTTTTTACTTTTCTAATAAACATGTTTGTTAAAACAGTTGGTTGAGTCTCTCATCAGTACCATGAAGTGAAAAGGAGGGAGATGGGTGCTAGTGAAAGGCAGGTCTCATCAGGCCCTGGGGTCCACCTGTGTGTTCCTCCATCCCTTTCCGGGACTGCCTGCCAGCCTGGTGACAGAGGTATGTGTACCGTGTGCCATCTGTGCCAAGAACCCTTGGCCCTGTGCTGGCTGAAGCTACTAGGAGAACACACTGGAAACCCCTGAAGGTAGGGGGTGAGTCAGACCTCCCACCGAGCCAGAGTCTCCAGAGGGAGAGCTCACCTGGCTGCCCCGGTTAGAAGGCCCAGACAGGAGGGCCCACTGAGCCCAAGGTCTTTAATAAACAGTGAAAGAACTAGAAAATACCTTTTTTCTTTCTGGGAGAAAAAAATCAGTTGTTCTACCATCATGAAAATTACAAATCTTATCTCTGGATTTAAAAAGACTAAGAGGGGGAGTTTGAAAAATGCTACTTAGCTCATGCCTCTCACTGGACAAGGAACCAAGAGGGGCATGAGCTGCCACTGGAAGGAAGTCTGGACCAGGCAGGTGCAGAACTCCTTGCACACACACAGGTGAGGGAAGGAGTGCTAAGGGCTTAGAGAGCAAGGAAATAGCAGGCCTACCTTAACGATACCTTGGGTCACCCAGACGTCCTCTGTGACCTGTTGTCCCAGCCACTTGGGATGCTGAAGCCAGAGGATTGCTTGAGTCCAGGAGTTCAAGGCTGCAGCCTGGGTGACAGCAAGACCCTGTCTCTAAAAACAAAAGATGCATTGTGCATTCTGGGCCCTAAGTTCCTTGTCAGAAAAATTGGCAAATTGGAGTTGGGCCAGATGAAGCCCAGGGCCTTTCCCCCTTCTTCCCTGGAATTACATCCACCCACACCAACTGGCATTTGGAGAGTTTTTATCTCCCTTGTTTCTCTTCCTGGTCCCAGACAGGCCTTGTTTGCTCAAGACTAAAACAGTGGCTCTTGCACTTGAGCGTGTGTAATCAGATCACATGGGCCCTCCTGGAGATTCTCAGTGCATCCGGGCCAGGGCCCCAGGAATCTGCATTTACATCAGGGATTCCAATAGAGGGTCCTAAACTTTACGATGACTTGGAAGACCCATCTCTCTTTGGTAGAACTGAGTGATTTGTTCCCCAAAATGGATAAACTTGGATTAGAAGTGCATTTGGAAATGGATAGTGAGAACAGTGAAGAAGGTGCCCAGACCAGTGTGAGGAAAGCCCTCACACCCATTTTCTTAGCACATCCTCCTAATAGCCACCCTGTGAGGGGTGGGTGGCGGCAGGGGTTTTTAATGTTTTACAGATTAGTAAATGGAGGCCCAAGCCCTGGAATGGTCATCTTCTTGGATAAGGGCTGACTTTGTCCACTCCCAGCCTATTGAGATTCAGGGCTGGGAGTAAAGCAAGCTTGACACCTGGGCTGAGTCTGAAGGGCACTTTCCCGGGAACTGAGATCTGTGCCAATGCTCACGACCCTGCCTCCCCAGTCTCAAAAGAGAAGCAAGGAAACAATTGTCTTCAGAGTACAGAGTCTTGTTGGTATTTGGGGGCCTGAGCTGGATGGATTGGTCAGCATTAGCCCTTCTCCAAGCCAAGGAGGCAGCTGAGCCCAGCACCGTACCTTTTCTTAGATAGTCAGACTGAGGCCAAATCAAAGCTTGGGTTCCCAAGCTAGCTCAGCTGAGATGCCCTCTCCCAGAGCCAAACTGGCTTCCCTCATTCTCACAGGGCCTGAAAATCCTCCTGCAGCAGGACAGAAGGGTCCGCAAGTGGGAGCCCTGAGCTTGAGTCTCACAGTAAAGGGAAGAGGGTTGCTGAGGCCCAAGACAGTTCTGGGTCCTGAGGTCTGCTAAGACATGGCAGGGAGCTTTGCCCCAGGTACCGGACTTGCTAAGAAAAAATGGGGCAGGCTTCTTTGTTGCTAAGCAGGAAAGCTAAGAAAGCTTATAAAGACCTACACATAAAATCAACTTTGGTAAATGCTTTCAGAAATTGTCCTATTTAGTGAGACCCTATCTGATAAAGAAAGGAAGGAAAGAAAAAAAAGAAAGGAAAGGAAAGGGGAAAGGAAAAAGGAAAGGAAAGGAAAAAGATTGCCCTATTTGGCCTGGAAAATATGTACTGATTACCTCCCTTCCTACAGGATTCCCTCTCTTATTCCTATTTCCCACCTCATCGGGTCAATGCTCTAGAACCTGCAATGGTTCCCTATTGCCAGATTCCACTGCTGGTCTTTTAGGACATTTGGTGTTGGGCCCTATACTCCCTACTGAGGCTTACCTCCCACCATTCCCCTTAACTGGGGGCAAGTCAATCTGGACTGCCCTTTTCTCTAGCCGCAGACTCCGCTCATTCTAGTGCCTTCCACCAAGGATATCTTCCTGAGTTTTGCTGGCAAGCCCTGTCCTTCCTTTAAGGCCTAGAGCAAGTTCCAGCCTTTTTATGGCTCTTTTCTGCTTTCCCTCCCCCCATCTTTCACAATTGCAGTGTGAAAACATGGCATACAATGATCTCATTGTATGCACTTGTTTTATATCCCCCTAGCTCTATAGAACACCCCTAGGTTCTCCTGAGGCCTGGCACATAGTGGTTAACTAAGCATTGGCTGTCCGAGCCTTGGGTCATGCCCTCTCTCTAAGCTCAGAGAACATAAGATTCCCTCACTAGCCTCAGACTCTCAGGCTACGAGGAGAGGAGGGCAGGGTTAGAAAGAGTTGTTGTTGTTTTTTTGTTTGTTTGTTTTTTGTTTTTTTCATTAGCCTACCACCTAGTTTAGAGGAATATTAAAGGCTAAGGTCCTGGAAGGGAGGTAGTGTCATTAGTGGCTAAAGCCACCTGCATTAGTGAAATCTGGATTCAGATCCCAGCTCAGCCATTTCCTAGCCAATGACCTGGGGCCAGTAACTTTACCTTCCTAAGCTTCAGTTTCCTCATCCGTAA'

In [ ]:
text = text[0:7000]

In [ ]:
ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id # A token used for prepending to the concatenated question-text word sequence

In [ ]:
lig = LayerIntegratedGradients(custom_forward, model.bert.embeddings)

input_ids, ref_input_ids, sep_id = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id)
token_type_ids, ref_token_type_ids = construct_input_ref_token_type_pair(input_ids, sep_id)
position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
attention_mask = construct_attention_mask(input_ids)

indices = input_ids[0].detach().tolist()
all_tokens = tokenizer.convert_ids_to_tokens(indices)

In [ ]:
# Check predict output
predict(input_ids)

tensor([[-0.6619,  0.5897]], grad_fn=<AddmmBackward0>)

In [ ]:
# Check output of custom_forward
custom_forward(input_ids)

tensor([0.7776], grad_fn=<UnsqueezeBackward0>)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
attributions, delta = lig.attribute(inputs=input_ids,
                                    baselines=ref_input_ids,
                                    n_steps=700,
                                    internal_batch_size=1,
                                    return_convergence_delta=True)

In [ ]:
score = predict(input_ids)

print('Sentence: ', text)
print('Sentiment: ' + str(torch.argmax(score[0]).cpu().numpy()) + \
      ', Probability positive: ' + str(torch.softmax(score, dim = 1)[0][1].cpu().detach().numpy()))

Sentence:  AAAGGTTTGGAGAGGGAGGTCCTAGTGAGGCTGAACAGAGGTAAAATAAACAGAAAGTTCCAGGCTCTCCTTGCTTCTACTTAAGAAAAATCAGAGCAGTAAATCCCAAACCAGTTCTGCCCACAACTCAAATGCAGATGAGCATCCCTTGCTCCATCTATGCAGGAAAGCCATAGTGTGGGGCTGTAGGACTATGGAGTCAGCCTCTGCAAGCAACAAGGCTCAGAGGCTGAGGGGCTTCAGGCCCCAGCAAAACCCAGTGATCCTGTTGTCAAATCTGAATGTCATCATCTGCCTCCAGGGCGACCAGCTCTGGATGGGGGTGTCAGGGTCAGGATCCTGTTCCATGTGGCAAGAGCTTGGGGAAGCTGCTGCAGGCCTACTGATGACCTTCTTTCCTGGCCCAGACACTCTTAAAAACTTCCTGGCGAGGGAGGCAACACACCTTATTGTGCAGCCCTCATCGAACGCATCAGCTGCACTAAGTGCTGAAGAAAAGTTCCCATAATAACCATGTGTGATAATAAAAAGATAAAGAGGCAAATAGGTATGTAGGTCAGCTCAAAAATAGCCACCCACCCGCCTGGCCTGGGTAGCCCTTTGTGCCAAGTGGGTAGAAAGGTTTTCCTTGCTTCCCTTCCCAAGCTGATGCCCCCTGCACTTTTCAAGGTTGCCTTAAATACAACTCCTTCCTGCACTAGCTGGGGTAGCCAGGGCAGTAGGGACTGCAGCCCTACAGGGGCAGCCCTGCGCTCCCTCCACCAAATCAATCCTGGGGTCAGAGGGCACAAAGGCTTGGCCTCTCTGACGTGAGTTCTCGGGGCTAAGTCAGGGTTTAGGGTAGTGAGTCACAGACATTCTTTCCCCCATCCCTTAGGGTTTGCGCACACCCAGGAAGCCTTCCCCTCCCCAACAGAGACCGCTTTGTTAGCACGACCCACCCGGTTGGAGAAGAGTTACAGCCTCAGGGGTGTGAGGGCCTCCTCGGC

In [ ]:
attributions_sum = summarize_attributions(attributions)

In [ ]:
# storing couple samples in an array for visualization purposes
score_vis = viz.VisualizationDataRecord(attributions_sum,
                                        torch.softmax(score, dim = 1)[0][1],
                                        torch.argmax(torch.softmax(score, dim = 0)[0]),
                                        1,
                                        text,
                                        attributions_sum.sum(),
                                        all_tokens,
                                        delta)

In [ ]:
print('\033[1m', 'Visualization For Score', '\033[0m')
viz.visualize_text([score_vis])

 Visualization For Score 


In [ ]:
df = dataframe_creation(attributions_sum, all_tokens, 74019545, 10)

In [ ]:
bed_file_creation(df, './67479')

Enter filename/content/drive/MyDrive/SE_annotation_project/bulk/67479


<ipython-input-7-0f8233eee82e>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_neg['score']= df_neg['score']*mult*(-1)
<ipython-input-7-0f8233eee82e>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pos['score']= df_pos['score']*mult


# SE_67637

In [ ]:
text = '...'

In [ ]:
text = text[0:10000]

In [ ]:
ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id # A token used for prepending to the concatenated question-text word sequence

lig = LayerIntegratedGradients(custom_forward, model.bert.embeddings)

input_ids, ref_input_ids, sep_id = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id)
token_type_ids, ref_token_type_ids = construct_input_ref_token_type_pair(input_ids, sep_id)
position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
attention_mask = construct_attention_mask(input_ids)

indices = input_ids[0].detach().tolist()
all_tokens = tokenizer.convert_ids_to_tokens(indices)

In [ ]:
predict(input_ids)

tensor([[-0.4575,  0.3728]], grad_fn=<AddmmBackward0>)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# Check output of custom_forward
custom_forward(input_ids)

tensor([0.6964], grad_fn=<UnsqueezeBackward0>)

In [ ]:
attributions, delta = lig.attribute(inputs=input_ids,
                                    baselines=ref_input_ids,
                                    n_steps=700,
                                    internal_batch_size=1,
                                    return_convergence_delta=True)

In [ ]:
score = predict(input_ids)

print('Sentence: ', text)
print('Sentiment: ' + str(torch.argmax(score[0]).cpu().numpy()) + \
      ', Probability positive: ' + str(torch.softmax(score, dim = 1)[0][1].cpu().detach().numpy()))

Sentence:  ACAGGACAGGAGTGAGGACAGGAGGTGGCCTGAGGGGGAGAAAGAGAAAGATTTTTGGCTTTATTGAGAATGGTTTGTGGAAGAATAAGAGGAAGCCAAGAAGAAGGAAAGCGAAACGCCCCCTCCCCCCATTGCAAGACAGGTTTCGGCTGTTTGAGACGGAGCATCACAGGCAGGAAGGAGGAGGCAGGGAGGGGGCCTGGGCTCCTTGCCCCCTGCTCGGCCCCCACCAAGAGGATTCCTGTTCTTAGCACTTTGGCCTTAGGGAGACTTGAGTTTCTTAGTCCGTGGGGAGGTTCCGTTCTCTGCCTTCACCACTCCATTTTCATGGTAACCTGGGAAGTGGAGAGAGACAAGCATACATGGTGTTTCTGCTGGAGCTAAGGCAGAAGAGGGGAGGGTGGGGCCAGGCCAAGGAAGGGGCCCGGGCAGCCCAGCTGCTATACATGGATGCACCAAAGGGGCGCCAGGCTGCAGCCTTGCACAGCAAATGCAAGCCAGCAGACGGCTCCCAGGAGGGTCCTGTCTTCCCCAGGGCCCTGGACTAGTGGCCCTGTTTGAGAGGCTTGGTCTCTGACTGCACTTTAGCAGGAGGGGCCTGGGATGACACACAGTAACTCAGCAACAACAGAGGCCAGCCTGCCCAAGCCTGAGGCTGGGGCTAGATACTCGCCGAAATCACTTAGATGCTGTCACTGCTCACTGTCCTCAGTTCTTCTCAGGCCCCTGCAAGGCATCAGCCCACTTCCAGTTCAGCAAGAACTCCCAGGAGCAGTGGCCACAGAAGGGAGGGGGACAAGAGGCTGTAGGAACAAGGCCTCAAGCAGGATGCCTGACACTCCTCTGGCAGAAAAATTGTGCTCTATTGTTCAAAATACCTCCGTGCGCTTGCTCAAATTCAGAGGTACTGGCTGCAGTCTTATGCCCCATACCTCCCCCGCCAGCCAGGCCTGACTCTGTGACACACCCAGCTGCGCGCGTGTCTGGTT

In [ ]:
attributions_sum = summarize_attributions(attributions)

In [ ]:
# storing couple samples in an array for visualization purposes
score_vis = viz.VisualizationDataRecord(attributions_sum,
                                        torch.softmax(score, dim = 1)[0][1],
                                        torch.argmax(torch.softmax(score, dim = 0)[0]),
                                        1,
                                        text,
                                        attributions_sum.sum(),
                                        all_tokens,
                                        delta)

In [ ]:
print('\033[1m', 'Visualization For Score', '\033[0m')
viz.visualize_text([score_vis])

 Visualization For Score 


In [ ]:
df = dataframe_creation(attributions_sum, all_tokens, 52367734, 6)

In [ ]:
bed_file_creation(df, './67637')

<ipython-input-18-b3c1f9b1a95a>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_neg['score']= df_neg['score']*mult*(-1)
<ipython-input-18-b3c1f9b1a95a>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pos['score']= df_pos['score']*mult


# SE_67718

In [7]:
text = 'AGCTACTTGGGAGGTTGAGGCAGGAGAATCACTTGAACCAGAAGGCAGAGGTTGCAGTGGGCCGAGATCACGTCATTGCACCCCAGCCTGGGCAACAAGAGGGAAACTCTGTCTCAAAAGAAAAAAAAAATTAACACAGGTAAAAAGCTTAGATCAGTGCTGAGCACAATATAAACCCATTAGTGGCTAAACCATCAACAAGTGTTGGGAGCTGCTGCTATTTTTACTTTCCCTAGAATTCTATCTCTTACCTTTCACTCTTCTCACTGCCCTTATTTGCCCAGAAATCCCTTTATGTCCTGAAAAACTCAACTCTCATATTTTACTTTTAGGAAATTATCAGTTACTGCTCTAACACAGTTCTTGATGTATGCTCTTCACTCCCTGACCTTTTGCTTTGTTGATACTTTATTATTGTACTTACGACTCCATACTTAAAAAATATAATTGGTAAAAAAAAATGTAACTAGTAATAGGCAGAAAAACTGGAATTGATTCCCAGTGCCTGTTACATGACAGGGGCTCATTATATACTTGCTGAGTGAATGATGGAAGGTCAAGTGAACAAATTAGAGAAATCAGGCAGAAAATATTTCCAGGTATCTTGGGGAGTTGAGAATTAGAGAAGAAAAGAATGAAGGAGGTTGGGATTTGATGAAGAACATATTTGTGACAGTGAAACTCAGTCGTGGGATTTGCTGTTTATATTGTTTTCTTTCCTTCTCAAATCAACATGAGTTGTACAGACCATAATAAAAGCTTCATCTTGAGCCTGTTATTGCATCACCATTTGGCTGATGTTTCACTGCTCCATCACTGTGGGAGGTTAACACTCACTGAGGGGTGGCAGTACCCTCTGCATTTCTCTGGGAAACTCCAATGATTCATCCAAAAGTGTGAGAAGCCAAGCAGCTTTTATCACCCAGGAGACAGCTGTAACCTGACCTCAATTCATGCCAAAGAAGCCTGAGGAAAGTGCCCTGGAAGTGAACAGCCTGCTCATGACTCAAGGAACTCAAAAGTCAGAGTTCTGACCAGTATTGTTGGGGTGGGTAAAAGAACTTCACTCCCTCAGATCTGAAAGTGTCCTTGCCAGAGGAGAGGGAGACCTTATTTCAAGTGTGTTTGAATAAACATGACATACATAAACTTGATTTTCCAGGTTTTGGAGAAAGCATACATTTGTCCTCTGCATCATGAAATAATGTCCTTGGTGGCTCCTTTCAGGGCCTGTGCTGCACTCTCCTTAGTGACTATTAGCATTGGAGTTGGACAATTTAAGGGCAGGAGTGGAGACCTGCTACTTATAAAGGATGACTACACTTATTTAGGGATACTGCTCACGCTGACTCTCAGAGCAATTCTTTTGTTCCAACAGCTGTTCCCAGGGTAGGGTAATATTGGAACAGAGGCTTATGCTGCAGTATCCATTGAGTGGGACTGAGGCCCCTTCAGTAGCTGAAAATCCCCACTTCAGTGCTACTGCTTCAGCTACTGCAGCTCTTACTGTCTTCAAAAACAATGGTAAGGATTCCTTTAACCTCAGAGGAGGAATTAGCTTTTTGCAGAGACTACAAAGAATCCATAGGATACAAACATTTCTATAGTCCAGGAAGGAGTCCCACATATAAAAACAAAACTGTAATATATTTTATAAGTGATTTGCCTAAATATGTGAGGCTGTATTCACTCAGTTTCTCAAGAAGCATTTCTTGAGAAGCTACTGTGTGCCAGAGCCTCCTTAGGGGGCAGGAACTGCTTATTTATCTTCATATCAACATCCCTAGGTAAGTGCCTAGCCCATCGTATGGGCTACAGACTATGCAATGCTATTTCCCTTAGCCCCACCAGAGTTCAGGGTCAGACTGATGGGCAGGACTATGAACAGATCTTCTTTCTCTCCTTGAAGGTAACTAACCTTCCATGGGACTTGAAGCTTCAGATTTTGGCCTTAACCAAGTCAACTAGCTCAAGTCAGAGGGGCAGTGATTATTTGAATTGCCAAGGTATTACCTAATGAGAATCTTTGTGACTAAGCTCTTAAAGTTGAAACTTAAGTAACATTAGCTGAAAAACTTGTTGGATAAACATTGCTACTCCCTTGAAAATTCCGGGAGTTCCAGTTTGGTTGCTGACCATTTTTTACCATTTATCTATTTGTTTAGCACTAACGTTTCTTAAGCTTCCAATGTATATAGAATGCTATGCTGGGTTGTGGGAAATACAATGAGCTATGCTCCCTGTCTGCAAGTCGGTCACATTTTAAGTGGGGGTGCAATAAATGACAGATTCAAACCACTGAACAATTATTAACAGAGAAAAGCAACTTTTATGGGCAAAGGCTTTCGGCAACTCCAGGCACAGTCTCACCCAAGGCCTTGATTGTGGCACTGCTTCTGTCACAGAGAATAAGAGTGAAATGTTAGAAGACTGTTGAAACACTGCGGAGTTAATGTTATGTTCCTTTTCATCTTTGGGTCTTTGTACATGCTGGTTCCTTGTCTTCCCACCACTACCCCTATATCATTTATCTAATTAAAACTCATAAACATGAATGAGAAGCCCTTTTTGACCACCAGCCTACAAACAGACACACAGACACACACACACACACACACACACACACACACACACACACTGCGTTGTCCTTCCATTGAGGTACCAACAGCACACTATAATTTCTCCATTATGATAGCATTTATTACAAGGTATTATGATTTCTGTTTAACTTCTCTGTATCCCTCCTGCCTGCTACATTGTAACTTCTGTGACTGCAGGGATATATCTGTTTTGTTCACCATTATATTGGCTAGCATGTAGCAGCCATTCAATAAATACTTGCTGAATGCATGAATGAACGGCTGAATGGCAGAACCAACATTTGAGTAAAATTTTCGTGTATTGTTTACTTTGTGCTATAACTTCCTAGGTATTTTATGGATCTTCTTTCTGTGGGGTTATATATAGTATAAATTTGTTGCAATATTATATTCTTTTTAAATGCCTCAGTCACTTGTTAAATTTAGTGCTTGTTGCAGAAGGGTATAATGTTACTGATAGGAAAGCCATAGTCTTGATCCCTATATCGAGGCCAAATGGCTTCTTATCTGGAAATCTCTGCCTTAAATTGAGCAGTATCTAGATTAGATGTTTTGAAACTATGTATCACTCACTATGGTGGGCAAATAGTTCAGACAGACTAGGAGACGGGATATGAGAAACTTAGAATAAGCTACAAGAAGAATTCTAAAGACACATGACCCATTAAATTCCTTTTTCATTGAAAATCAGACCAAAATCTGTGGCAAAAACAGAAAACGTATTCCGGAAATATGGCTGTAGAACTGAAGACTAAATAGCAAGCAGTGTTTAATCTGAAGTAATGAATGTTATTTCAAGGAGCAAAGAAGCTAACACTTGCAATAAATCTTTGTTTTGAAGTATCTTTCTCAAAGGTCATCAAAATAAGTTTTAATTTAGATTCCTAAATCCAGTTTCTGCTCTGTTAGTTATTGGGGATTTTTTGAATAAATAATCTACGCAGAACTGTGGACTCTGCTCTGCTTTTTCAACCCCATGGTGTGGTCATCCAGAAAGGATTTCTTCTCCGCTCTAGGCTTAGAAATGGGGTACCTCCTGCCTGATGTATCCTATGCTTTCCAGAACCTGAAAGATTAAGATGAATCATTACAAGTGAAATGACCCTAGACAGAGAACAAAGAGGGCTGGCACATGTTTGCTGTAACATTCCGGGAGAGCTCTTCATCCCGTTTACATGTTCTTGAGAGGGTGCCTTCCGTGCTGTGATGAATGAGCCACCCAGACCTCCCGCTACCACACAGCAATTGTTTAGTCAGAGATCCTGTGCATCTGGGTCTTTTTTTTAATCAGCAGGTTCTAAATAAAAAACAAAAAACAATAACAAGCCCACCCTCTCTGAATATGACTCTCTGACTCTCTTGTGTTTCTTGTTTACATTTTATTGTGCGAATGTGATTCTGCTTCTTAGCCTAGTGCATTTGGATTTGTTGTTCACACTGAGCATAAATTGCCAAACACTCAACTTTAATCTAATTGTACATACTATAAACAGAAAATGTTGACGTTAAAAGCATTTCCATAATCACAGATTGGGTGTATGGATGAGCAAGTGGATGAGTTAGGCCTTTTGTCATCTCAGTCCATAGCTTGATCCTTCAGGTATTCAAAAGGAAACGGGGACTGTCTTCCACTGATGACCTCACTGATGATGAGGTTCATCTGCCACCATCTATAGTGAGTGAAACTTTATGGTGTAGTTGTTAAGTCTCAAATTCAGAGAGACTTAGGATCAAATCTCACTCTGCTACTTTCTAGCTATGTGACCTTGGGAAAGCCACATGGTGCCTTGAAGACTCAGTCTCCTTAACTATATTATAAGAATAATAATATTCCCTATGCCAAAGGGATCCTGTGAGATAATAACATCTTAATTGAGTTAAATTATTTAAGAGTTAAATGAGATAATGGATAAAAATCATGGCACATTTTGAGCACTTCATAAATATTAATGATAACGATCCTTATTTAAACGTGCCCTGCATTCTAAAAATGAGAAGTAACTTTCTGGCTGAAACCAACCCCTGAATGGCAACAGAGAAAGCTGTTTCAACAGGGTTGAGAAAAAAGCTGGGGGATAAAGTAATGTTTCTAATCTTAGAGTTTAAGAGAAAATACATTTTTTAATTCACTTGGTAAATTAGAGATCTTTTAGGGACACCAGAAGAAGGCACTTTCTGAAAGAGTTAAAATGATTTCCTTAGAGTAAACAAATCTCTCTCATAGAATTTTTGCTTAGACTAAGTTGATGGGACATTCATTAAGAAATAAACTTTTCATCCTCGCTTGGGTAATGCAACCAGGGAACGAGTAAATATCCCGGGAAAGCCAACTTGCTATATGCAGGTACTATTTCTTAAAAATGTTATTTTTATCTATCAAAATAATATTACTGTTATTAAGTATCATTGAGGTTCAATCCTGCGACAGAAACTGTACTTAGCTATAGGGAATCAAATATGATTGTGACATAGTTCTACCCTCAGGGGTTTTAAGTGGGAAGATAGAATATGCCATTAAAAAAAAAGTAATCCTTGGCATGGAAGGTTAAATGCCTTATAACTAACTGATGATTGATAGCAGAAGAGTGACTAATTCTCATCTAGAAAGAGAAGGCCTGGGCTCTAGTTTTGGTTCCACCACTTGTCAAGTTTGTAACCTAGGGAAATTCACTTCAATCCTTTGAACTTTCAGTTTTTCAAAGGAAATGGAGATGACGATATTTGCCCTACCATTCCCATAAGGTTGCTGTGAGGTTCAAATGAGCTAAGGTTTACGAAAACACATAATAAATTACAGCACTTCGGTGCTATTCCTTGGTGTGATCATTTTCCTTCCTCTGCCTCCTCATAGACTGAGACTCTCATAAACTGTTTTATTTTCTGTCTCAGCTTATGAAATATCTGCAAGTTATATCATTATGTGTGTGTGTGTATTGTGTGCACATTATCAAGTCTAATCAAGCCAAAGCAGCAGAACTAAACACAGATATCTGGACTCCTAAAAATCCACCACTTAATCGTGAAGACAACATTGTTAGAGAAGAAAAGATATTCTCTAAGGTTGGACCTATTTCAGAAATGAGTAGCATCACCCTTGCTTTACAATGGAGAAAACTGAGTATTGATTTATTGAACACTTATTTAATAAAGGTTTATGAAGAACCCATCCTGCACCAGGGGCCCAGGTACCAACTTAAAGAATGTGAGTGAGAAAAACAGGGAACTCAGTCCTGGCATTGACCTTTGCTGTAACTTCTGCTCATTGTAGGAATCAGCCAACTTCCTGTCCTTTAGTTTCCTCTTCTGCAAACTGAAGATTAGTTAGTTGTAGAGCATACAGTCAGTGTTATGTCTGAAAATATTGAAATACAGAGACAAAGTGAAAAAGAAGGGTATGAAACCCCTTTCCATTAGACTGTAAGAGGGATGATGATTGTTACTTTTGGGAGTTATGAGCGTGGGCAGTGGAGAGAGTGGTTTGTTTGAGGATGCCACGCTGAGCATTGGGTGGATGTGGAATTTGTGTGGGCAGGGAAGTAGAGAAGAGAGTTGGGGCAAGCTAAGACTGCCAGCTTCCAGACTGGAAGCTTCTGCGGTAGTTACGGGGAATGAGTCAGGAAATAGTTCTGTGTAAATAGTTATGAAATTTTAAAGCTAATTTTGTCATGGGCCAAAGGCATTTTTTGGGGATGATGAGAGATATCAAATTTTGTGGATGCGCCTACAAGTAGGTGATGGTTCTACACTGAAAGATTTGCCACTTTTCTTCACCCTAGAATGTTCATATTGTGCGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTATGGTTTCTTAGTGGACTGTGAGATATATTAAACATAGATAAAAGGAATATTTAACTAAAAGAGAAGCAACAGAAAGTTAATCAAATAGTATCCAAGACAGGACGGAAATGGGCTTTAGGGGAGTCCTGAGAGAAAGAGAACTCACATTCATTGAGTTCGTGTGAATTGTTTCTGGGCCCCGGGCACTGTCATATCCATGAAATAACCTTATTCTTGTCTTGAAAAGCTGTTCTCATACAGATCTGAAAACTAAGGCCCAGAGACAGTAAGCAGCTTCTGGGAATGACTATGTTGTTGAGCTCAGGTTTGAGCTATGGAGGGTATGAAGTTATTTCCTCTTAGCAACTCCATCACAGACTCAACATGCTGGGCAAACTCACTTTATCTTTAAAGAACAGTCTCTATTTGCAAAGGAGAGAACCTGCTTCCATACACTTGGGACTGTTAGAGTTGTCAGACGATGTTACCAAAGCAAAGGCACTTGAACTTAGGCCAAGGTATATGTTTCTGTAGTCACCCTGTGGTAACTACTGGGGTCAGCATTTCACAAAACCTACTTACCCTGTTTAGTTAGGAAAACCCTAGAGGTGGGGGAGTGGGAGTGAGGGTCCCCAGTCCCACTGTCATTTGGAAAAATGTCCAGATGAGAGTAGGCTGTCCACTTCCTAGAGGCAAATCTTTCTGAATCTACATAGTTTCACAGACTCACAGAACTTCAGAGCTGAAGAGACCTTCAGATAGATATTTTTCAGATAGACCTTCAGATAGACATTTTGCCGCACTGCTCAGTCTAGGTTAACGGGAAGCTAGGGCAAGAATTCACTTATGGCCCTCCCTTCTTCCTGTATTCAGTATTTTCCTGTCTGTAACATTGGGTCCATGATTAAGCTATTTAACCAGTTTCTGAATTTGTAATTTATGAAGCCAGGAAAAGGGCATTTATCACTTGAATCATCCCAGCAGCCTGTGTCAGAGTTAAGGAGGGAATTATTTTTTTTTCTAAAAATTCTGTTCTAGGCTATTTAAGGTCACTCTACTTGCCTCTTCTGGAGAGACTCGTACATGCGTTGCTGATCCCTGGGCTGAGTTCTCCCAGCCCTAGTGAACTCGGAAGTACAGGTCCAGAAGGTTCTGCTTGATGGTTCCCATCAGGGTAAAACTGAAAATAGGAAGCCGTTGGTTGCTGATTGTAAGATAAATTTGTTTAAGTAGTGTGCCAAAAGCAGTGGGTCATATGAAAAGCCACATGACCAACCAGAAATAATTGGAGCTTAAAAGATGCTAGGGTCTATGAGTAAGATTTCTTTCACCATGGGATTTAGGTAAGGCAAACTCACCCACTCACTAGTTCTTAATTCCCATTTTCTTTTTGTCCTGGAACATTTCACCTGCCCTGATGCGGCAAATTAAGTGATTGAATTAATACAGAAATGTCAGCCTTTATTGATGACAATTAGGTGTTCATATTATTTCTGTTTTATTAATAATTTATTTACCTATTCAACAAATCTTTAAAGCCTACTGTGTGCTAGATGTGGAGGAAGAAACAAGTAACACTTCCTCCGTGACAGTTTTTCTAATCCCTCTAGTGTAAATTCTCTCTCCCTTTCCTGAAACTTAGTAGTTCTTTGTTTATAGTTCTCCAATAGATTTATGCCATGTTGCCTTGTATTGTAGCAGCTTGAGAACATGTCTTACTATTTAGAACTAAGACAATTGCTTTACAAGTATTTGTCTGTACCATGATATTTAAAGTAGTGCTGGCACAGAGTATGTTGTCAGTGTTGTGGAATAAATGACTAAATGAATATTTGTTGAATTAGGTAAGAAGTTGCTTCAGCTTTGGCTGTTATGCTGCTGTAACAAACAGCTCCCAAATTTCAATAGCTCAGAACACAATGATTTATTTCTCATCTATGTTAATGCCCATAGTGAGTTAGGTGCAGCTCTGCTCCCCATCTTCTGGATTTGGGGATCTGGGTAGAAATAGCCATTAGTTGGGATTTTCTTGTGTTGGGGAACAAGAGAGATGGCAGAAACACATGATAGATCAGAAGTGGCATTCATTGGCTGGGCACGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCATAGGCAGGTGGATCACCTGAGCTCAGGAGCCCAAGATCAGCCTGGCCAACAATGGTGAAACCCCATCTCTACTAAAAATACAAAAATTAGCCAAGCACGGTGGCAGGCACCTGTATCTCAGCTACTTGGAAGGCTGAGGCAAGAGAATTGCTTGAACCCGGGAGGTGGAGGTTGCAGTGAGCTAAGATTGCGCTACCACCCTCCACCTGAGCAGCAGAGCGAGACTCTGTCAAAAAACAAACAAACAAACAAACAACATCACTTCTGCTCAAATTTCGTTGGCCCAAACTAGTCATGTAGCCACGCCTGCCGTGGAGGGCAGACAAGGATAATTCTCCCATGAGAAAGGCCATGAATCTTTGGGAATAATAATATAATTTACCAGAGAAGTCATCCTTATTTTTTAAAGAACTTAAAATTTAGTTGAGGAAGTAAGCTATACACAAAGAAATACAATAAAAAAAAGGTGCAAGCAACATGTTATAGAGGTGAGGGAGGAGAGATTTCTTTGGACTAAGATTAGTCACAGAAGGATTAATAAAAGATGTGTTCTTTAAATTGGACATGAAAAGTAAGATTTTGCCAAGTGGAACTGGACAAGAGGTGAATAGAAGAGTATTTAAAAAGATATGAAGGTACTCAAACAAAAAATGTACTAAAAAAAAGAGTGAAAAGTCCACTTTTGGTGGATAATTAGGTCCACATTTCAAAGTTGGGAATTTAGAGTCTATTAAGCAGGAAATTATAAACCACTAGAGATTTGGAAAGCTAACTAAAGGATGCAATGGAGAGAAGCTGAAAATATTTTTGAGGCTATGCAAAGAGTTCAGGAAACAGGGACTAAAGACAGAATCAGAATAGTGAAAAGTATGAACAGGACAGAACATACAGAGGCAAGATGCAAAGGAAATGTACAAAGGTTGAAAACCGATTGGTTTGGAAAGCAAAGGAGAGAGGCAGTGAAAGAAGTTGCAGATGATTATTCCATTAATGGTGGTGCCATTAATGGAATTCAAAAAATAAGAAGAGGACTGACTCATAGGAAGAGAAAATAAGTTTTGTTTGAGGTATTCTGATTTCGAGGAATTTCAGAGAGAAATGTTTAGTAGGCGTATGGAACTCTGGGAATAGAGTTTGTGGGATTGTGGTTGTGATTGAAGATGTAGTCCTGAGGGACCCCTGAAGAGGTGACAGTTGAAGCCATGAGTGCAGATGAGAATCCCAAAGGAAACTAGTGGCCAAAAGGCAGAGTCTTGGAGGGCTGCCAACATGTAGGATGTGGGAAGATGAAGACAAACCAGGGAGGGAAGCTGAAGAACCTTGACAAGGCAGAGAGGAGAACACTATAGTGTGGGATCAAGTGAGCTGAGGCAGGAGAGAGGCTGAGGAGCAGGGAGAGGTCAGCGGTGTCAAATGCTTTAGAAAGTGCTAAGAAGATGAGAATGTAGGAAGTGGTATTGAGGTTAGAGTAGGAGATCATTGGCGATGAGTAAGGAAACAACTTCAGTGAGGACTGGAGATGTTCTTGTGAGATATCTGATAACAAAAATTACAGAGATTGCATAATGGAGGATTAGTTGGAACTGGTGGAACTGAGAGTACGCTTAGTGCGATAACCTTTGAAAAGAAGAGGAACCTATTCCATTTGGGGAAGGCAGGAGAAAACTATGCACTAAGAGAGAACTTGTCCATGAGAAGGAGTGATTTTTAAGGTGCTTCTTAACAAAGTAGAGTTTAGGGAATGTATTCACTCTTTGAAATTAAAAAGCAATTGCTTATTATTGAAAACCCAGGTTAGCTATTAAAAAAAAAAAAAGAGAAAAATAAGACAAAAAAAGAAAAAAAAACCATGCATGTGTAACAAGTCAGGGGAAGCTGAGGAGACCTTCCAAACTGTACTGTATTCACTTGCATATATTCCACATCTGGCACTGGGCAGTCAAATATTCACAGGTAATAAATAGATATTAAAGAATGTATTATTGGGCATTAGGACTCATAATTTTAGAAAAATGTTTTTTGTGGCTGGGCGTGGTGGCTCACTCCTGTAATCCCAGCACTTTGGGAGGCCAAGGAGAGTGGATCAGAAGGTCAGGAGATCGACAGCATCCTGGCCAACATGGTGAAACCCCGT'

In [8]:
ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id # A token used for prepending to the concatenated question-text word sequence

lig = LayerIntegratedGradients(custom_forward, model.bert.embeddings)

input_ids, ref_input_ids, sep_id = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id)
token_type_ids, ref_token_type_ids = construct_input_ref_token_type_pair(input_ids, sep_id)
position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
attention_mask = construct_attention_mask(input_ids)

indices = input_ids[0].detach().tolist()
all_tokens = tokenizer.convert_ids_to_tokens(indices)

In [9]:
predict(input_ids)

tensor([[-0.1817,  0.0391]], grad_fn=<AddmmBackward0>)

In [10]:
torch.cuda.empty_cache()
custom_forward(input_ids)

tensor([0.5550], grad_fn=<UnsqueezeBackward0>)

In [11]:
attributions, delta = lig.attribute(inputs=input_ids,
                                    baselines=ref_input_ids,
                                    n_steps=700,
                                    internal_batch_size=1,
                                    return_convergence_delta=True)

score = predict(input_ids)

print('Sentence: ', text)
print('Sentiment: ' + str(torch.argmax(score[0]).cpu().numpy()) + \
      ', Probability positive: ' + str(torch.softmax(score, dim = 1)[0][1].cpu().detach().numpy()))

attributions_sum = summarize_attributions(attributions)

# storing couple samples in an array for visualization purposes
score_vis = viz.VisualizationDataRecord(attributions_sum,
                                        torch.softmax(score, dim = 1)[0][1],
                                        torch.argmax(torch.softmax(score, dim = 0)[0]),
                                        1,
                                        text,
                                        attributions_sum.sum(),
                                        all_tokens,
                                        delta)

Sentence:  AGCTACTTGGGAGGTTGAGGCAGGAGAATCACTTGAACCAGAAGGCAGAGGTTGCAGTGGGCCGAGATCACGTCATTGCACCCCAGCCTGGGCAACAAGAGGGAAACTCTGTCTCAAAAGAAAAAAAAAATTAACACAGGTAAAAAGCTTAGATCAGTGCTGAGCACAATATAAACCCATTAGTGGCTAAACCATCAACAAGTGTTGGGAGCTGCTGCTATTTTTACTTTCCCTAGAATTCTATCTCTTACCTTTCACTCTTCTCACTGCCCTTATTTGCCCAGAAATCCCTTTATGTCCTGAAAAACTCAACTCTCATATTTTACTTTTAGGAAATTATCAGTTACTGCTCTAACACAGTTCTTGATGTATGCTCTTCACTCCCTGACCTTTTGCTTTGTTGATACTTTATTATTGTACTTACGACTCCATACTTAAAAAATATAATTGGTAAAAAAAAATGTAACTAGTAATAGGCAGAAAAACTGGAATTGATTCCCAGTGCCTGTTACATGACAGGGGCTCATTATATACTTGCTGAGTGAATGATGGAAGGTCAAGTGAACAAATTAGAGAAATCAGGCAGAAAATATTTCCAGGTATCTTGGGGAGTTGAGAATTAGAGAAGAAAAGAATGAAGGAGGTTGGGATTTGATGAAGAACATATTTGTGACAGTGAAACTCAGTCGTGGGATTTGCTGTTTATATTGTTTTCTTTCCTTCTCAAATCAACATGAGTTGTACAGACCATAATAAAAGCTTCATCTTGAGCCTGTTATTGCATCACCATTTGGCTGATGTTTCACTGCTCCATCACTGTGGGAGGTTAACACTCACTGAGGGGTGGCAGTACCCTCTGCATTTCTCTGGGAAACTCCAATGATTCATCCAAAAGTGTGAGAAGCCAAGCAGCTTTTATCACCCAGGAGACAGCTGTAACCTGACCTCAATTCATGCCAAAGAAGCCTGAGGAAAGTGCCCTGGAAGTG

In [12]:
print('\033[1m', 'Visualization For Score', '\033[0m')
viz.visualize_text([score_vis])

 Visualization For Score 


In [13]:
df = dataframe_creation(attributions_sum, all_tokens, 101086907, 1)
bed_file_creation(df, './67718')

<ipython-input-6-b3c1f9b1a95a>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_neg['score']= df_neg['score']*mult*(-1)
<ipython-input-6-b3c1f9b1a95a>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pos['score']= df_pos['score']*mult
